In [1]:
from pyomo.environ import *

# Sample Data
AssetMean = [0.0101110, 0.0043532, 0.0137058]
AssetCovar = [[0.00324625, 0.00022983, 0.00420395],
              [0.00022983, 0.00049937, 0.00019247],
              [0.00420395, 0.00019247, 0.00764097]]
num_assets = len(AssetMean)
select_count = 2  # For your case, this will be 4

# Model
model = ConcreteModel()

# Variables
model.x = Var(range(num_assets), within=NonNegativeReals)  # weights
model.y = Var(range(num_assets), within=Binary)  # binary selection variables

# Objective
def objective_rule(m):
    return sum(m.x[i] * AssetMean[i] for i in range(num_assets))
model.objective = Objective(rule=objective_rule, sense=maximize)

# Constraints
def weight_sum_rule(m):
    return sum(m.x[i] for i in range(num_assets)) == 1
model.weight_sum = Constraint(rule=weight_sum_rule)

def select_asset_rule(m, i):
    return m.x[i] <= m.y[i]
model.select_asset = Constraint(range(num_assets), rule=select_asset_rule)

def number_of_assets_rule(m):
    return sum(m.y[i] for i in range(num_assets)) == select_count
model.number_of_assets = Constraint(rule=number_of_assets_rule)

# Solving
solver = SolverFactory('ipopt')
solver.solve(model)

# Output
for i in range(num_assets):
    print(f"Asset {i}: Selected={model.y[i]()}, Weight={model.x[i]()}")



ipopt


ApplicationError: No executable found for solver 'ipopt'